In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# data preprocessing and augmentation
data_dir = "D:/Percia_MTech/GUVI/python/Projects/covid19_detection/data"   # dataset folder containing train/test folders

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform)
test_data  = datasets.ImageFolder(os.path.join(data_dir, "test"),  transform=test_transform)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=16, shuffle=False)

class_names = train_data.classes
print("Classes:", class_names)


Classes: ['Covid', 'Normal', 'Viral Pneumonia']


In [ ]:
# cnn model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64*56*56, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

model = SimpleCNN(num_classes=3)


In [ ]:
# transfer learning model
model = models.resnet50(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, 3)  # 3 classes

In [5]:
# train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(5):
    model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")


Epoch 1 Loss: 0.3501
Epoch 2 Loss: 0.0320
Epoch 3 Loss: 0.0529
Epoch 4 Loss: 0.0121
Epoch 5 Loss: 0.0613


In [ ]:
# --- SAVE MODEL ---
torch.save(model.state_dict(), "model.pth")
print("Model Saved as model.pth ✅")

In [6]:
model.eval()
preds, truths = [], []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds.extend(torch.argmax(outputs, 1).cpu().numpy())
        truths.extend(labels.numpy())

print(classification_report(truths, preds, target_names=class_names))
print(confusion_matrix(truths, preds))


                 precision    recall  f1-score   support

          Covid       1.00      1.00      1.00        26
         Normal       1.00      0.70      0.82        20
Viral Pneumonia       0.77      1.00      0.87        20

       accuracy                           0.91        66
      macro avg       0.92      0.90      0.90        66
   weighted avg       0.93      0.91      0.91        66

[[26  0  0]
 [ 0 14  6]
 [ 0  0 20]]
